#### **Assignment 7 🧠📈👾🦾** 

#### **Student Name:** Nadeen Moreb 👩🏻‍💻

#### **Student ID:** 1203437 

# Quality of Sleep 🌜💤

## Data Set description

#### The "Sleep Health and lifestyle" dataset contains features and metrices related to sleep and the factors and daily habits that affect its quality for individuals with different ages.

it consists of 374 rows (persons) and 13 different columns (variables). 

these columns include: 
1. **Person ID**: person's id. 

2. **Gender**: person's gender, which is either a male or a female

3. **Age**: person's age

4. **Occupation**: person's professional occupation/job

5. **Sleep Duration (hours)**: person's daily sleep hours

6. **Quality of Sleep (scale: 1-10)**: person's sleep quality that is in a range from 1 to 10

7. **Physical Activity Level (minutes/day)**: person's daily physical activity duration in minutes.

8. **Stress Level (scale: 1-10)**: person's stress level that is in a range from 1 to 10

9. **BMI Category**: person's BMI category (Underweight, Normal, Overweight)

10. **Blood Pressure (systolic/diastolic)**: person's blood pressure (systolic pressure over diastolic pressure)

11. **Heart Rate (bpm)** : person's resting heart rate measured in beats per minute.

12. **Daily Steps** : person's daily steps count.

13. **Sleep Disorder**: does this person suffer any kind of the sleep disorders, and has one of the values (None, Insomnia, Sleep Apnea).

#### In this notebook, I will analyze and predict quality of sleep and how it is affected by factors inclduing sleep metrices, lifestyle, health and cardiovascular health, and sleeping disorders existence factors using linear regression machine learning techniques and metrices. 

### importing numpy, pandas, seaborn, matplot and sklearn libraries 

In [76]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn import set_config
set_config(transform_output='pandas')


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/sleep-health-and-lifestyle-dataset/Sleep_health_and_lifestyle_dataset.csv


## Loading The Data

In [77]:
df = pd.read_csv("/kaggle/input/sleep-health-and-lifestyle-dataset/Sleep_health_and_lifestyle_dataset.csv")

In [78]:
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


### 1. Perform necessery data cleaning and preprocessing, the splitting of the data should use 80% train and 20% test ratio. Use your student ID as the random state when splitting.

#### Steps for Data preprocessing:

#### - Removing non relevant or constant features 

which is the Person ID since it doesn't give any value to what I want to predict in this ML model 

In [80]:
df = df.drop(columns='Person ID')

#### - Verifying data types

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender                   374 non-null    object 
 1   Age                      374 non-null    int64  
 2   Occupation               374 non-null    object 
 3   Sleep Duration           374 non-null    float64
 4   Quality of Sleep         374 non-null    int64  
 5   Physical Activity Level  374 non-null    int64  
 6   Stress Level             374 non-null    int64  
 7   BMI Category             374 non-null    object 
 8   Blood Pressure           374 non-null    object 
 9   Heart Rate               374 non-null    int64  
 10  Daily Steps              374 non-null    int64  
 11  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(6), object(5)
memory usage: 35.2+ KB


#### - investigating columns 

In [82]:
df['Gender'].value_counts()

Gender
Male      189
Female    185
Name: count, dtype: int64

gender is a nominal categorical feature that consists of 2 values (with their counts): 189 Males or 185 Females

In [83]:
df['Occupation'].value_counts()

Occupation
Nurse                   73
Doctor                  71
Engineer                63
Lawyer                  47
Teacher                 40
Accountant              37
Salesperson             32
Software Engineer        4
Scientist                4
Sales Representative     2
Manager                  1
Name: count, dtype: int64

occupation is a nominal categorical feature that consists of 11 different values that include(with their count): 73 Nurses, 71 Doctors, 63 Engineers, 47 lawyers, 40 Teachers, 37 Accountants, 32 Salespersons, 4 Software Engineers, 4 Scientists, 2 Sales Representatives, and 1 Manager.

In [84]:
df['BMI Category'].value_counts()

BMI Category
Normal           195
Overweight       148
Normal Weight     21
Obese             10
Name: count, dtype: int64

BMI Category is an ordinal categorical feature that consists of 3 different values, where each has the following count: 195 Normal and 21 Normal Weight (both refers to same class and therefore should be handled), 148 overweight, and 10 Obese. 

In [85]:
df['Blood Pressure'].value_counts()

Blood Pressure
130/85    99
140/95    65
125/80    65
120/80    45
115/75    32
135/90    27
140/90     4
125/82     4
132/87     3
128/85     3
126/83     2
115/78     2
139/91     2
142/92     2
119/77     2
135/88     2
129/84     2
128/84     2
131/86     2
117/76     2
130/86     2
118/75     2
121/79     1
122/80     1
118/76     1
Name: count, dtype: int64

blood pressure is a nominal categorical value that consists of 25 different values that include(with their counts): 99 130/85, 65 140/95, 65 125/80, 45 120/80, 32 115/75, 27 135/90, 4 140/90, 4 125/82, 3 132/87, 3 128/85, 2 126/83, 2 115/78, 2 139/91, 2 142/92, 2 119/77, 2 135/88, 2 129/84, 2 128/84, 2 131/86, 2 117/76, 2 130/86, 2 118/75, 1 121/79, 1 122/80, 1 118/76. I will handle this column by dividing it into two columns, Systolic and Diastolic columns and convert it to numeric type.

In [86]:
df['Sleep Disorder'].value_counts()

Sleep Disorder
Sleep Apnea    78
Insomnia       77
Name: count, dtype: int64

Sleep disorder is a nominal categorical value that consists of 3 different values, where each has the following count: 78 Sleep Apena, 77 Insomnia, and Nan values(which refers to the No disorder value and should be handled). 

In [87]:
df['Age'].value_counts()

Age
43    34
44    30
37    20
38    20
50    20
31    18
32    17
53    17
59    16
39    15
45    14
33    13
30    13
29    13
35    12
36    12
41    12
49    11
57     9
52     9
42     9
51     8
54     7
58     6
28     5
40     4
48     3
55     2
56     2
34     2
27     1
Name: count, dtype: int64

In [88]:
print(df['Age'].max())
print(df['Age'].min())

59
27


Age is an integer numerical value that ranges between 27 and 59

In [89]:
df['Sleep Duration'].value_counts()

Sleep Duration
7.2    36
6.0    31
7.8    28
6.5    26
6.1    25
7.7    24
6.6    20
7.1    19
8.1    15
7.3    14
8.4    14
6.3    13
8.5    13
8.0    13
6.2    12
8.2    11
7.6    10
6.4     9
7.9     7
7.4     5
6.7     5
7.5     5
6.8     5
8.3     5
5.9     4
6.9     3
5.8     2
Name: count, dtype: int64

In [90]:
print(df['Sleep Duration'].max())
print(df['Sleep Duration'].min())

8.5
5.8


Sleep Duration is a float numerical value that ranges between 5.8 and 8.5 

In [91]:
df['Quality of Sleep'].value_counts()

Quality of Sleep
8    109
6    105
7     77
9     71
5      7
4      5
Name: count, dtype: int64

In [92]:
print(df['Quality of Sleep'].max())
print(df['Quality of Sleep'].min())

9
4


Quality of Sleep is an integer numerical value that ranges between 4 and 9

In [93]:
df['Physical Activity Level'].value_counts()

Physical Activity Level
60    70
30    68
45    68
75    67
90    67
40     6
55     6
35     4
50     4
70     3
42     2
32     2
80     2
65     2
85     2
47     1
Name: count, dtype: int64

In [94]:
print(df['Physical Activity Level'].max())
print(df['Physical Activity Level'].min())

90
30


Physical Activity is an integer numerical value that ranges between 30 and 90

In [95]:
df['Stress Level'].value_counts()

Stress Level
3    71
8    70
4    70
5    67
7    50
6    46
Name: count, dtype: int64

In [96]:
print(df['Stress Level'].max())
print(df['Stress Level'].min())

8
3


Stress Level is an integer numerical value thatranges between 3 and 8. 

In [97]:
df['Heart Rate'].value_counts()

Heart Rate
68    94
70    76
72    69
65    67
75    36
78     5
85     3
80     3
84     2
83     2
73     2
67     2
74     2
77     2
81     2
76     2
69     2
86     2
82     1
Name: count, dtype: int64

In [98]:
print(df['Heart Rate'].max())
print(df['Heart Rate'].min())

86
65


Heart Rate is an integer numerical value that ranges between 65 and 86.

In [99]:
df['Daily Steps'].value_counts()

Daily Steps
8000     101
6000      68
5000      68
7000      66
10000     36
5500       4
3000       3
3500       3
4000       3
6800       3
4800       2
7300       2
7500       2
4200       2
3300       2
5600       2
5200       2
4100       2
3700       2
6200       1
Name: count, dtype: int64

In [100]:
print(df['Daily Steps'].max())
print(df['Daily Steps'].min())

10000
3000


Daily Steps is an integer numerical value that ranges between 3000 and 10000

#### - handling inconsistencies in dataframe 
by replacing 'Normal Weight' with 'Normal' in the 'BMI Category' column

In [101]:
df['BMI Category'] = df['BMI Category'].replace('Normal Weight', 'Normal')

split 'Blood Pressure' column into two seperate numeric columns, 'Systolic'and 'Diastolic', and dropping 'Blood Pressure' . 

In [102]:
# split the columns
df[['Systolic Blood Pressure', 'Diastolic Blood Pressure']] = df['Blood Pressure'].str.split('/', expand=True)

# convert to numeric value
df['Systolic Blood Pressure'] = pd.to_numeric(df['Systolic Blood Pressure'])
df['Diastolic Blood Pressure'] = pd.to_numeric(df['Diastolic Blood Pressure'])

# drop the original column
df.drop(columns=['Blood Pressure'], inplace=True)

#### - Removing duplicates

In [103]:
df.duplicated().sum()

242

In [104]:
df = df.drop_duplicates()
df.duplicated().sum()

0

now data frame is cleaned from duplicates 

#### - feature matrix and Target vector I want to predict 

In [105]:
# The Target 
y = df["Quality of Sleep"]

# The features used to make the prediction
X = df.drop(columns = "Quality of Sleep")
X

,Gender,Age,Occupation,Sleep Duration,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Sleep Disorder,Systolic Blood Pressure,Diastolic Blood Pressure
0,Male,27,Software Engineer,6.1,42,6,Overweight,77,4200,NaN,126,83
1,Male,28,Doctor,6.2,60,8,Normal,75,10000,NaN,125,80
3,Male,28,Sales Representative,5.9,30,8,Obese,85,3000,Sleep Apnea,140,90
5,Male,28,Software Engineer,5.9,30,8,Obese,85,3000,Insomnia,140,90
6,Male,29,Teacher,6.3,40,7,Obese,82,3500,Insomnia,140,90
...,...,...,...,...,...,...,...,...,...,...,...,...
358,Female,59,Nurse,8.0,75,3,Overweight,68,7000,NaN,140,95
359,Female,59,Nurse,8.1,75,3,Overweight,68,7000,NaN,140,95
360,Female,59,Nurse,8.2,75,3,Overweight,68,7000,Sleep Apnea,140,95
364,Female,59,Nurse,8.0,75,3,Overweight,68,7000,Sleep Apnea,140,95


#### - Splitting the data into train and test

random_state equals my student ID number which is 1203437

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1203437)

X_train

,Gender,Age,Occupation,Sleep Duration,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Sleep Disorder,Systolic Blood Pressure,Diastolic Blood Pressure
93,Male,35,Lawyer,7.4,60,5,Obese,84,3300,Sleep Apnea,135,88
37,Male,31,Doctor,7.6,75,6,Normal,70,8000,NaN,120,80
52,Male,32,Doctor,6.0,30,8,Normal,72,5000,NaN,125,80
53,Male,32,Doctor,7.6,75,6,Normal,70,8000,NaN,120,80
222,Male,44,Salesperson,6.3,45,7,Overweight,72,6000,Insomnia,130,85
...,...,...,...,...,...,...,...,...,...,...,...,...
167,Male,41,Lawyer,7.1,55,6,Overweight,72,6000,NaN,125,82
109,Male,37,Lawyer,7.4,60,5,Normal,68,8000,NaN,130,85
366,Female,59,Nurse,8.1,75,3,Overweight,68,7000,Sleep Apnea,140,95
358,Female,59,Nurse,8.0,75,3,Overweight,68,7000,NaN,140,95


### -Categorical features Lists

In [107]:
# this contains all categorical features, including nominal and ordinal
categorical_columns = make_column_selector(dtype_include='object')

# BMI ordinal features 
ordinal_columns = ['BMI Category']

# Nominal features
one_hot_columns = X_train[categorical_columns].drop(columns=ordinal_columns).columns


### -Numerical features List

In [108]:
# numerical features
numerical_columns = make_column_selector(dtype_include='number')

#### - Handling missing values

In [109]:
df.isna().sum()

Gender                       0
Age                          0
Occupation                   0
Sleep Duration               0
Quality of Sleep             0
Physical Activity Level      0
Stress Level                 0
BMI Category                 0
Heart Rate                   0
Daily Steps                  0
Sleep Disorder              73
Systolic Blood Pressure      0
Diastolic Blood Pressure     0
dtype: int64

Sleep Disorder column has 73 null values, firstly I will investigate this column using value_counts() method

In [110]:
df['Sleep Disorder'].value_counts()

Sleep Disorder
Sleep Apnea    30
Insomnia       29
Name: count, dtype: int64

the output shows that people with no sleeping disorders are filled as null in the Sleep Disorder columns, hence we handle them by replacing each with the constant value: "No Disorder" using simple imputer with the nominal pipeline

### - Categorical Pipelines

### - Nominal Pipeline

In [111]:
# Nominal pipeline

#imputer for filling the null values in the Sleep Disorder column with No Disorder value
impute_disorders_na = SimpleImputer(strategy='constant', fill_value='No Disorder')

# one hot encoder Instance 
one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# pipeline with the imputer and the one hot encoder
one_hot_pipe = make_pipeline(impute_disorders_na, one_hot_encoder)

# one hot pipe and columns tuple
one_hot_tuple = ('categorical', one_hot_pipe, one_hot_columns)



### - Ordinal Pipeline 

In [112]:
df['BMI Category'].value_counts()

BMI Category
Normal        73
Overweight    52
Obese          7
Name: count, dtype: int64

In [113]:
# Ordinal pipeline

# the order of categories in the BMI Category ordinal column
BMI_order = ["Normal", "Overweight", "Obese"]

# BMI order List
ordinal_category_orders = [BMI_order]

# ordinal encoder instance, where the list of ordered values is passed
ordinal_encoder = OrdinalEncoder(categories=ordinal_category_orders)

# ordinal scaler 
ordinal_scaler = StandardScaler()

# ordinal pipeline with the ordinal encoder and ordinal scaler
ordinal_pipe = make_pipeline(ordinal_encoder, ordinal_scaler)

# ordinal pipe and columns tuple
ordinal_tuple = ('ordinal', ordinal_pipe, ordinal_columns)

### - Numerical Pipeline 

In [114]:
# numerical scaler 
scaler = StandardScaler()

# numerical pipeline with the numerical scaler
numerical_pipe = make_pipeline(scaler)

# numerical pipe and columns tuple
numerical_tuple = ('numerical',numerical_pipe, numerical_columns)

### - ColumnTransformer

In [115]:
# ColumnTransformer Instance
# with verbose_feature_names_out=False 
#and the numerical, ordinal, and one hot tuples are passed
column_transformer = ColumnTransformer([numerical_tuple, ordinal_tuple, one_hot_tuple],
                                    remainder='passthrough',
                                    verbose_feature_names_out=False)
column_transformer

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical',
                                 Pipeline(steps=[('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x784a02cd1720>),
                                ('ordinal',
                                 Pipeline(steps=[('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Normal',
                                                                              'Overweight',
                                                                              'Obese']])),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['BMI Category']),
                                ('categorical',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='No '
                                                                           'Disorder',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['Gender', 'Occupation', 'Sleep Disorder'], dtype='object'))],
                  verbose_feature_names_out=False)

### - Fitting train data 

In [116]:
column_transformer.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical',
                                 Pipeline(steps=[('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x784a02cd1720>),
                                ('ordinal',
                                 Pipeline(steps=[('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Normal',
                                                                              'Overweight',
                                                                              'Obese']])),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['BMI Category']),
                                ('categorical',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='No '
                                                                           'Disorder',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['Gender', 'Occupation', 'Sleep Disorder'], dtype='object'))],
                  verbose_feature_names_out=False)

### - Transform train and test data

In [117]:
# train data transform
X_train_processed = column_transformer.transform(X_train)

# test data transform
X_test_processed = column_transformer.transform(X_test)

### - Resulted processed data

In [118]:
X_train_processed.head()

,Age,Sleep Duration,Physical Activity Level,Stress Level,Heart Rate,Daily Steps,Systolic Blood Pressure,Diastolic Blood Pressure,BMI Category,Gender_Female,...,Occupation_Manager,Occupation_Nurse,Occupation_Sales Representative,Occupation_Salesperson,Occupation_Scientist,Occupation_Software Engineer,Occupation_Teacher,Sleep Disorder_Insomnia,Sleep Disorder_No Disorder,Sleep Disorder_Sleep Apnea
93,-0.637305,0.423034,0.117142,-0.327753,2.725120,-1.953662,0.887532,0.602229,2.507058,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
37,-1.076979,0.687263,0.883470,0.262202,-0.225674,0.801001,-1.053536,-0.745131,-0.869796,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
52,-0.967061,-1.426572,-1.415513,1.442112,0.195868,-0.957294,-0.406513,-0.745131,-0.869796,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
53,-0.967061,0.687263,0.883470,0.262202,-0.225674,0.801001,-1.053536,-0.745131,-0.869796,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
222,0.351961,-1.030228,-0.649185,0.852157,0.195868,-0.371196,0.240510,0.096969,0.818631,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


processed data: 

- Person Id is ommitted, hence only 12 columns left
- No duplicates
- no null data, specifically in the sleep disorder column. 
- the numerical data is scaled.
- also, ordinal BMI Category is now a numerical column that is also scaled. 
- one hot columns for nominal columns are added, including the gender, occupation and sleep disorders, which has increased the overall number of training data columns to 25 (13 new columns, since it was previously 12 columns without the Person ID column). 

### 2. Fit a linear regression model on the transformed data. 

In [119]:
# linear regression model instance
linear_regression_model = LinearRegression()

# combining the linear regression model and the preprocessing ColumnTransformer in a Pipeline
linear_regression_model_pipe = make_pipeline(column_transformer, linear_regression_model)

# linear_regression_model_pipe
linear_regression_model_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numerical',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x784a02cd1720>),
                                                 ('ordinal',
                                                  Pipeline(steps=[('ordinalencoder',
                                                                   OrdinalEncoder(categories=[['Normal',
                                                                                               'Overweight',
                                                                                               'Obese']])),
                                                                  (...
                                                                   StandardScaler())]),
                                                  ['BMI Category']),
                                                 ('categorical',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='No '
                                                                                            'Disorder',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  Index(['Gender', 'Occupation', 'Sleep Disorder'], dtype='object'))],
                                   verbose_feature_names_out=False)),
                ('linearregression', LinearRegression())])

linear regression model is now fitted and trained to the processed data columns 

### 3. Print the model parameters and provide an interpretation for each

In [120]:
feature_names = column_transformer.get_feature_names_out()
print(f"features: {feature_names}")

features: ['Age' 'Sleep Duration' 'Physical Activity Level' 'Stress Level'
 'Heart Rate' 'Daily Steps' 'Systolic Blood Pressure'
 'Diastolic Blood Pressure' 'BMI Category' 'Gender_Female' 'Gender_Male'
 'Occupation_Accountant' 'Occupation_Doctor' 'Occupation_Engineer'
 'Occupation_Lawyer' 'Occupation_Manager' 'Occupation_Nurse'
 'Occupation_Sales Representative' 'Occupation_Salesperson'
 'Occupation_Scientist' 'Occupation_Software Engineer'
 'Occupation_Teacher' 'Sleep Disorder_Insomnia'
 'Sleep Disorder_No Disorder' 'Sleep Disorder_Sleep Apnea']


In [121]:
coefs = linear_regression_model.coef_.round(2)
print(f"coffiecients or weights: {coefs}")

coffiecients or weights: [ 0.25  0.29 -0.05 -0.7  -0.14  0.2   0.47 -0.46 -0.02 -0.13  0.13  0.77
  0.21  0.05  0.34  0.13  0.25 -0.74 -0.21 -0.77  0.1  -0.14 -0.11  0.07
  0.04]


In [122]:
intercepts = linear_regression_model.intercept_.round(2)
print(f"intercept or bias: {intercepts}")

intercept or bias: 6.92


In [123]:
# final model equation
eq = " +\n".join([ f"{'' if i == 0 else ' '*20}{coefs[i]} * {feature_names[i]}"  for i in range(len(feature_names))])
print(f"Model equation: y_hat = {eq}")

Model equation: y_hat = 0.25 * Age +
                    0.29 * Sleep Duration +
                    -0.05 * Physical Activity Level +
                    -0.7 * Stress Level +
                    -0.14 * Heart Rate +
                    0.2 * Daily Steps +
                    0.47 * Systolic Blood Pressure +
                    -0.46 * Diastolic Blood Pressure +
                    -0.02 * BMI Category +
                    -0.13 * Gender_Female +
                    0.13 * Gender_Male +
                    0.77 * Occupation_Accountant +
                    0.21 * Occupation_Doctor +
                    0.05 * Occupation_Engineer +
                    0.34 * Occupation_Lawyer +
                    0.13 * Occupation_Manager +
                    0.25 * Occupation_Nurse +
                    -0.74 * Occupation_Sales Representative +
                    -0.21 * Occupation_Salesperson +
                    -0.77 * Occupation_Scientist +
                    0.1 * Occupation_Software Engine

#### Age Interpret = 0.25

When Age increases by one year, sleep quality is expected to increase by 0.25, while assuming that the other features' values are constant and do not change.

#### Sleep Duration Interpret = 0.29
When Sleep Duration increases by one hour, sleep quality is expected to increase by 0.29, while assuming that the other features' values are constant and do not change.


#### Physical Activity Level = -0.05
When daily Physical Activity engagement increases by 1 minute, sleep quality is expected to decrease by 0.05, while assuming that the other features' values are constant and do not change.

                      

#### Stress Level = -0.7
When Stress Level rating/scale increases by 1, sleep quality is expected to decrease by 0.7, while assuming that the other features' values are constant and do not change.


#### Heart Rate = -0.14 
When the resting Heart rate increases by 1 beat per minute, sleep quality is expected to decrease by 0.14, while assuming that the other features' values are constant and do not change.
                     

#### Daily Steps = 0.2
When the number of Daily Steps increases by one, sleep quality is expected to increase by 0.2, while assuming that the other features' values are constant and do not change.             

#### Systolic Blood Pressure  = 0.47
When Systolic Blood Pressure increases by one millimeter of mercury (mmHg)(which is the measurement unit of blood pressure), sleep quality is expected to increase by 0.47, while assuming that the other features' values are constant and do not change.  

#### Diastolic Blood Pressure  = -0.46 
When Diastolic Blood Pressure increases by one millimeter of mercury (mmHg), whihc is the measurement unit of blood pressure, sleep quality is expected to decrease by 0.46, while assuming that the other features' values are constant and do not change.  
                       

#### BMI Category  = -0.02
When BMI Category ordinal level increases by one, meaning that the person weight is increased, sleep quality is expected to decrease by 0.02, while assuming that the other features' values are constant and do not change.  

#### Gender_Female = -0.13
if the gender of the person is female, sleep quality is expected to decrease by 0.13, while assuming that the other features' values are constant and do not change.              

#### Gender_Male = 0.13
if the gender of the person is male, sleep quality is expected to increase by 0.13, while assuming that the other features' values are constant and do not change.   

#### Occupation_Accountant = 0.77
if Occupation is an Accountant, sleep quality is expected to increase by 0.77, while assuming that the other features' values are constant and do not change. 

#### Occupation_Doctor = 0.21
if Occupation is a Doctor, sleep quality is expected to increase by 0.21, while assuming that the other features' values are constant and do not change. 

#### Occupation_Engineer = 0.05
if Occupation is an Engineer, sleep quality is expected to increase by 0.05, while assuming that the other features' values are constant and do not change. 

#### Occupation_Lawyer = 0.34
if Occupation is a Lawer, sleep quality is expected to increase by 0.34, while assuming that the other features' values are constant and do not change. 

#### Occupation_Manager = 0.13
if Occupation is a Manager, sleep quality is expected to increase by 0.13, while assuming that the other features' values are constant and do not change. 

#### Occupation_Nurse = 0.25
if Occupation is a Nurse, sleep quality is expected to increase by 0.25, while assuming that the other features' values are constant and do not change. 

#### Occupation_Sales Representative = -0.74
if Occupation is a Sales Representative, sleep quality is expected to decrease by 0.74, while assuming that the other features' values are constant and do not change. 

#### Occupation_Salesperson = -0.21
if Occupation is a Salesperson, sleep quality is expected to decrease by 0.21, while assuming that the other features' values are constant and do not change. 

#### Occupation_Scientist = -0.77
if Occupation is a Scientist, sleep quality is expected to decrease by 0.77, while assuming that the other features' values are constant and do not change. 

#### Occupation_Software Engineer = 0.1
if Occupation is a Software Engineer, sleep quality is expected to increase by 0.1, while assuming that the other features' values are constant and do not change. 

#### Occupation_Teacher = -0.14
if Occupation is a Teacher, sleep quality is expected to decrease by 0.14, while assuming that the other features' values are constant and do not change. 

#### Sleep Disorder_Insomnia = -0.11
if a person suffers from an Insomnia Sleep Disroder, sleep quality is expected to decrease by 0.11, while assuming that the other features' values are constant and do not change. 

#### Sleep Disorder_No Disorder = 0.07
if a person suffers from No Sleep Disroder at all, sleep quality is expected to increase by 0.07, while assuming that the other features' values are constant and do not change. 

#### Sleep Disorder_Sleep Apnea = 0.04
if a person suffers from an Apena Sleep Disroder, sleep quality is expected to increase by 0.04, while assuming that the other features' values are constant and do not change. 

### 4. Evaluate the model's quality using appropriate metrices. 

first lets have a glimpse at and overview the min, max, and average values of the target variable Quality of Sleep

In [124]:
df.describe()

,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps,Systolic Blood Pressure,Diastolic Blood Pressure
count,132.000000,132.000000,132.000000,132.000000,132.000000,132.000000,132.000000,132.000000,132.000000
mean,41.128788,7.082576,7.151515,58.393939,5.537879,71.204545,6637.878788,128.363636,84.537879
std,8.813942,0.775335,1.269037,20.468840,1.740428,4.867306,1766.288657,7.825650,6.049926
min,27.000000,5.800000,4.000000,30.000000,3.000000,65.000000,3000.000000,115.000000,75.000000
25%,33.750000,6.400000,6.000000,44.250000,4.000000,68.000000,5000.000000,120.750000,80.000000
50%,41.000000,7.150000,7.000000,60.000000,6.000000,70.000000,7000.000000,130.000000,85.000000
75%,49.000000,7.725000,8.000000,75.000000,7.000000,74.000000,8000.000000,135.000000,88.500000
max,59.000000,8.500000,9.000000,90.000000,8.000000,86.000000,10000.000000,142.000000,95.000000


As shown: 
- the **min** value = **4.0**
- the **max** value = **9.0**
- the **average*** values = the mean = **7.312834**

by finding metrices mae, mse, rmse, and r squared for both train and test data because using only test data to check quality is not enough:

In [125]:
# training data predictions
y_train_pred = linear_regression_model_pipe.predict(X_train)

# calculating metrics
mae = mean_absolute_error(y_train, y_train_pred)
mse = mean_squared_error(y_train, y_train_pred)
rmse = mean_squared_error(y_train, y_train_pred, squared=False) 
r_squared = r2_score(y_train, y_train_pred)
print(f"Regression Metrics: Training Data:")
print(f"- MAE = {mae:,.3f}")
print(f"- MSE = {mse:,.3f}")
print(f"- RMSE = {rmse:,.3f}")
print(f"- R^2 = {r_squared:,.3f}")        


# test data predictions 
y_test_pred = linear_regression_model_pipe.predict(X_test)

# calculating metrics
mae = mean_absolute_error(y_test, y_test_pred)
mse = mean_squared_error(y_test, y_test_pred)
rmse = mean_squared_error(y_test, y_test_pred, squared=False) 
r_squared = r2_score(y_test, y_test_pred)
print('\n' + f"Regression Metrics: Test Data:")
print(f"- MAE = {mae:,.3f}")
print(f"- MSE = {mse:,.3f}")
print(f"- RMSE = {rmse:,.3f}")
print(f"- R^2 = {r_squared:,.3f}")      


Regression Metrics: Training Data:
- MAE = 0.211
- MSE = 0.093
- RMSE = 0.305
- R^2 = 0.943

Regression Metrics: Test Data:
- MAE = 0.234
- MSE = 0.129
- RMSE = 0.359
- R^2 = 0.913


by comparing errors for test and train data, I see that there is no gap between both, hence no overfit, therefore I can check for model's quality by overviewing and analysing each of the errors. 

compared to the quality of sleep range that is between 4.0 and 9.0 and mean value that is 7.312834, these errors' values are relatively low, indicating a good quality model, also, the R squared metric that is above 0.9 is also high, in which indicates to an accurate model 

Additionally, the meaning of each of the Test data regression metrices values is as following: 
- **MAE = 0.234**: the average of the absolute errors values, in other words, the absolute differences between the actual and the predicted values, it tells us that on average, quality of sleep predictions are off by 0.234 unit from the actual values, which is acceptable.



- **MSE = 0.129**: the average of the squared differences between the actual and the predicted quality of sleep values is 0.129. 


- **RMSE = 0.359**: the square root of the MSE is 0.359, meaningt that the predictions are close by 0.359 to the quality of sleep true values on average. 


- **R^2 = 0.913**: my model can explain 0.913 of the variance in quality of sleep variable. which is close to 1, meaning its ability to predict values that are close to real values is good and accurate.

### 5. Determine if the model exhibits overfitting or underfitting and explain your reasoning.

no overfitting 

by comparing errors for test and train data: 

Regression Metrics: Training Data:
- MAE for Training Data = 0.211 and MAE for Test Data = 0.234, which is close. 
- MSE for Training Data = 0.093 and MSE for Test Data = 0.129, which is also close. 
- RMSE for Training Data = 0.305 and RMSE for Test Data = 0.359 are close to each other as well. 

I see that there is no gap between errors for both Training Data and Testing data, hence no overfit. 


### Kaggle Dataset Link:  
#### https://www.kaggle.com/datasets/uom190346a/sleep-health-and-lifestyle-dataset